In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
# import seaborn as sns
import pickle
import time
import gc
from tqdm import tqdm, tqdm_notebook

%matplotlib inline

#每次可以输出多个变量
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from pylab import rcParams
rcParams['figure.figsize'] = 14, 6

import warnings
warnings.filterwarnings("ignore")

#中文字体
import matplotlib
matplotlib.use('qt4agg')
#指定默认字体
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['font.family'] = 'sans-serif'
#解决负号'-'显示为方块的问题
matplotlib.rcParams['axes.unicode_minus'] = False

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test_public.csv')

train.shape
train.head()
test.shape
test.head()

(9947, 5)

,content_id,content,subject,sentiment_value,sentiment_word
0,vUXizsqexyZVRdFH,因为森林人即将换代，这套系统没必要装在一款即将换代的车型上，因为肯定会影响价格。,价格,0,影响
1,4QroPd9hNfnCHVt7,四驱价格貌似挺高的，高的可以看齐XC60了，看实车前脸有点违和感。不过大众的车应该不会差。,价格,-1,高
2,QmqJ2AvM5GplaRyz,斯柯达要说质量，似乎比大众要好一点，价格也低一些，用料完全一样。我听说过野帝，但没听说过你说...,价格,1,低
3,KMT1gFJiU4NWrVDn,这玩意都是给有钱任性又不懂车的土豪用的，这价格换一次我妹夫EP020可以换三锅了,价格,-1,有钱任性
4,nVIlGd5yMmc37t1o,17价格忒高，估计也就是14-15左右。,价格,-1,高


(2364, 2)

,content_id,content
0,XuPwKCnA2fqNh5vm,欧蓝德，价格便宜，森林人太贵啦！
1,2jNbDn85goX3IuPE,楼主什么时候提的车，南昌优惠多少啊
2,hLgEADQ8sUnvGFK9,吉林，2.5优惠20000，送三年九次保养，贴膜
3,nZmM7LQsfr03wUaz,便宜2万的豪华特装，实用配制提升，优惠还给力，确实划算。
4,pwd8MnrthDqLZafe,如果实在想买就等车展期间，优惠2万，我24.98万入的2.5豪


In [3]:
train.groupby(
    ['subject'], as_index=False)['content_id'].agg({
        'subject_num': 'count'
    })
train.groupby(
    ['sentiment_value'], as_index=False)['content_id'].agg({
        'sentiment_value_num':
        'count'
    })

,subject,subject_num
0,价格,1273
1,内饰,536
2,动力,2732
3,外观,489
4,安全性,573
5,操控,1036
6,油耗,1082
7,空间,442
8,舒适性,931
9,配置,853


,sentiment_value,sentiment_value_num
0,-1,1616
1,0,6661
2,1,1670


In [4]:
test['subject'] = '暂无'
test['sentiment_value'] = 99

In [5]:
df = pd.concat([train, test], axis=0)
df.reset_index(drop=True, inplace=True)

df.shape

(12311, 5)

## 分词及停用词

In [6]:
import jieba
import re

In [7]:
stopwords = []
with open('stopwords/chinese.txt', 'r', encoding='utf-8') as f:
    for line in f:
        stopwords.append(line[:-1])
stopwords = list(set(stopwords))
len(stopwords)

1472

In [8]:
# 停用词、数字、字母
def sent2words(sentence):
    sentcut = jieba.cut(sentence)
    words = []
    for w in sentcut:
        if (re.compile(r'[^\w]').findall(w) == []) and (
                w not in stopwords) and (
                    re.compile(r'-?([1-9]\d*\.\d*|0\.\d*[1-9]\d*|0?\.0+|0)$')
                    .findall(w) == []) and (
                        re.compile(r'^[A-Za-z0-9]+$').findall(w) == []):
            words.append(w)
    return words

## 词典

In [9]:
import operator
from functools import reduce
from collections import Counter
from gensim import corpora

In [10]:
words = []

for sentence in tqdm_notebook(df.content):
    words.append(sent2words(sentence))

A Jupyter Widget

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ZERO\AppData\Local\Temp\jieba.cache
Loading model cost 0.617 seconds.
Prefix dict has been built succesfully.


In [11]:
dictionary = corpora.Dictionary(words)

In [157]:
# words = reduce(operator.add, words)
# word_num = Counter(words)
# len(word_num)
# sorted(dict(word_num).items(), key=lambda x:x[1], reverse=False)
# for k, v in dict(word_num).items():
#     if v == 1:
#         del word_num[k]
# len(word_num)

In [158]:
with open('feature/dict(noStopnoNumLetter).txt', 'w', encoding='utf-8') as f:
    for word in words:
        _ = f.write(' '.join(word) + '\n')

## Word2Vec
- 全部
- 无停用词
- 无字母数字
- 无停用词和字母数字

In [159]:
import gensim

In [24]:
sentcut_len = []

for sentence in tqdm_notebook(df.content):
    sentcut_len.append(len(sent2words(sentence)))

A Jupyter Widget

In [27]:
Counter(sentcut_len)  # max:79

Counter({1: 2,
         2: 32,
         3: 161,
         4: 387,
         5: 514,
         6: 563,
         7: 586,
         8: 680,
         9: 689,
         10: 627,
         11: 639,
         12: 595,
         13: 573,
         14: 500,
         15: 477,
         16: 464,
         17: 419,
         18: 402,
         19: 352,
         20: 318,
         21: 258,
         22: 276,
         23: 249,
         24: 236,
         25: 198,
         26: 197,
         27: 187,
         28: 123,
         29: 137,
         30: 134,
         31: 122,
         32: 91,
         33: 102,
         34: 69,
         35: 99,
         36: 68,
         37: 45,
         38: 65,
         39: 33,
         40: 46,
         41: 40,
         42: 48,
         43: 38,
         44: 27,
         45: 30,
         46: 35,
         47: 43,
         48: 22,
         49: 33,
         50: 21,
         51: 30,
         52: 17,
         53: 18,
         54: 30,
         55: 6,
         56: 23,
         57: 26,
         58: